In [ ]:
from model import layers, activations, networks, utils, optimizers, losses, paddings, metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
import random
import os
from numpy import asarray

In [ ]:
def loadImages(n):
    filepath = 'datasets\\catsanddogs\\Petimages'
    list_x = []
    list_y = []

    c = 0
    for category in ['Cat', 'Dog']:
        path = filepath + '\\' + category
        for i, image in enumerate(os.listdir(path)):
            if i >= n / 2: break
            if image[-3:] != 'jpg': continue
            image_rgb = Image.open(path + '\\' + image)
            image_resized = image_rgb.resize((64, 64))
            image_arr = np.asarray(image_resized)
            if len(image_arr.shape) != 3: continue
            list_x.append(image_arr / 255)
            list_y.append(0 if category == 'Cat' else 1)
            c = c + 1
            print(f'{c}/{n} images loaded', end='\r')
    return np.array(list_x), np.array(list_y)

X, Y = loadImages(2000)

In [ ]:
Y = np.expand_dims(Y, -1)
print(X.shape)
print(Y.shape)

In [ ]:
X, Y = utils.shuffle(X, Y)
l = int(len(X) * 0.7)
[x_train, x_test] = np.split(X, [l])
[y_train, y_test] = np.split(Y, [l])

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
model = networks.FeedForward([
    layers.Input(input_shape=(64, 64, 3)),
    layers.Convolution(nr_kernels=1, kernel_size=(3, 3), activation=activations.ReLu, padding=paddings.Zero, stride=1),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Flatten(),
    layers.Dense(nr_neurons=16, activation=activations.ReLu),
    layers.Dense(nr_neurons=1, activation=activations.Sigmoid),
    layers.Output()
    ])

In [ ]:
model.compile(
    optimizer=optimizers.stochastic_gradient_descent(learning_rate=0.001, momentum=0.9, nesterov=True),
    loss=losses.root_mean_square,
    metric=metrics.Accuracy
    )

In [ ]:
for l in model.layers:
    print (l.output.shape)

In [ ]:
model.train(x_train, y_train, epochs=20, batch_size=200)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
i = random.randint(0, len(X) - 1)
image = X[i]
plt.imshow(image)

Cat=0, Dog=1

In [ ]:
p = model.predict(image)
p

In [ ]:
plt.imshow(model.layers[1].output[:, :, 0], cmap='gray')

In [ ]:
plt.imshow(model.layers[2].output[:, :, 0], cmap='gray')